### Dataset

In [1]:
import numpy as np
import tensorflow as tf

digit_to_word_map = {
    0: 'pad', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 
    5: 'five', 6: 'six', 7: 'seven', 8: 'eight', 9: 'nine'
}

even_sentences = []
odd_sentences = []
seqlens = []
for i in range(10000):
    rand_seq_len = np.random.choice(range(3,7))
    seqlens.append(rand_seq_len)
    rand_odd_ints = np.random.choice(range(1,10,2), rand_seq_len)
    rand_even_ints = np.random.choice(range(2,10,2), rand_seq_len)    
    
    if rand_seq_len < 6:
        rand_odd_ints = np.append(rand_odd_ints, [0]*(6-rand_seq_len))
        rand_even_ints = np.append(rand_even_ints, [0]*(6-rand_seq_len))        
        
    even_sentences.append(' '.join([digit_to_word_map[r] for r in rand_even_ints]))
    odd_sentences.append(' '.join([digit_to_word_map[r] for r in rand_odd_ints]))    
    
data = even_sentences + odd_sentences
seqlens *= 2

word2index_map = {}
index = 0
for sent in data:
    for word in sent.lower().split():
        if word not in word2index_map:
            word2index_map[word] = index
            index += 1

index2word_map = {index: word for word, index in word2index_map.items()}
vocabulary_size = len(index2word_map)

labels = [1]*10000 + [0]*10000
for i in range(len(labels)):
    label = labels[i]
    one_hot_encoding = [0]*2
    one_hot_encoding[label] = 1
    labels[i] = one_hot_encoding
    
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]

labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]

train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]

test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]

import zipfile
import numpy as np
import tensorflow as tf

path_to_glove = 'data/glove.840B.300d.zip'
PRE_TRAINED = True
GLOVE_SIZE = 300
batch_size = 128
embedding_dimension = 64
num_classes = 2
hidden_layer_size = 32
time_steps = 6

def get_glove(path_to_glove, word2index_map):
    embedding_weights = {}
    count_all_words = 0
    
    with zipfile.ZipFile(path_to_glove) as z:
        with z.open('glove.840B.300d.txt') as f:
            for line in f:
                vals = line.split()
                word = str(vals[0].decode("utf-8"))
                
                if word in word2index_map:
                    count_all_words += 1
                    coefs = np.asarray(vals[1:], dtype='float32')
                    coefs /= np.linalg.norm(coefs)
                    embedding_weights[word] = coefs
                    
                if count_all_words == vocabulary_size - 1:
                    break   
                    
    return embedding_weights

word2embedding_dict = get_glove(path_to_glove, word2index_map)
embedding_matrix = np.zeros((vocabulary_size, GLOVE_SIZE))
for word, index in word2index_map.items():
    if not word == "pad":
        word_embedding = word2embedding_dict[word]
        embedding_matrix[index,:] = word_embedding
        
data_indices = list(range(len(data)))
np.random.shuffle(data_indices)
data = np.array(data)[data_indices]
labels = np.array(labels)[data_indices]
seqlens = np.array(seqlens)[data_indices]
train_x = data[:10000]
train_y = labels[:10000]
train_seqlens = seqlens[:10000]
test_x = data[10000:]
test_y = labels[10000:]
test_seqlens = seqlens[10000:]

def get_sentence_batch(batch_size, data_x, data_y, data_seqlens):
    instance_indices = list(range(len(data_x)))
    np.random.shuffle(instance_indices)
    batch = instance_indices[:batch_size]
    x = [[word2index_map[word] for word in data_x[i].split()] for i in batch]
    y = [data_y[i] for i in batch]
    seqlens = [data_seqlens[i] for i in batch]
    return x, y, seqlens

# print(even_sentences[0:6])
# print(odd_sentences[0:6])
# print(seqlens[0:6])

x, y, seqlens = get_sentence_batch(20, train_x, train_y, train_seqlens)

In [2]:
import tensorflow as tf

batch_size = 20
lstm_size = 10

with tf.name_scope('input'):
    x = tf.placeholder(tf.int32, shape=(None, 6))
    y = tf.placeholder(tf.int32, shape=(None, 2))
    seqlens = tf.placeholder(tf.int32, shape=(None,))    
    emb = tf.nn.embedding_lookup(embedding_matrix, x)
    
with tf.name_scope('lstm'):
    cells = [tf.nn.rnn_cell.LSTMCell(lstm_size) for _ in range(2)]
    lstm_cell = tf.nn.rnn_cell.MultiRNNCell(cells)
    
    outputs, (cell_state, states) = tf.nn.dynamic_rnn(
        lstm_cell, emb, sequence_length=seqlens, 
        dtype=tf.float64
    )
    state = states[-1]
    
with tf.name_scope('output'):
    logits = tf.layers.dense(state, 2)
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=y,
        logits=logits,
        name='loss'
    ))
    correct = tf.equal(
        tf.argmax(y, 1), 
        tf.argmax(logits, 1)
    )
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    
    train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(500):
        batch_x, batch_y, batch_seqlens = get_sentence_batch(
            batch_size, train_x, train_y, train_seqlens
        )
        
        sess.run(train_step, {
            x: batch_x,
            y: batch_y,            
            seqlens: batch_seqlens
        })
        
        if step % 100 == 0:
            ce, acc = sess.run([cross_entropy, accuracy], {
                x: batch_x,
                y: batch_y,            
                seqlens: batch_seqlens
            })
            print('Train Cross Entropy: %f, Accuracy: %f' % (ce, acc))
        
    test_x, test_y, test_seqlens = get_sentence_batch(100000, test_x, test_y, test_seqlens)
        
    ce, acc = sess.run([cross_entropy, accuracy], {
        x: test_x,
        y: test_y,            
        seqlens: test_seqlens
    })
    print('Test Cross Entropy: %f, Accuracy: %f' % (ce, acc))

Train Cross Entropy: 0.690684, Accuracy: 0.650000
Train Cross Entropy: 0.121787, Accuracy: 1.000000
Train Cross Entropy: 0.009364, Accuracy: 1.000000
Train Cross Entropy: 0.005921, Accuracy: 1.000000
Train Cross Entropy: 0.002550, Accuracy: 1.000000
Test Cross Entropy: 0.001505, Accuracy: 1.000000
